In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import pylab
import scipy.stats as ss
import csv
import numpy as np
import sklearn
import glob
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [ ]:
#df = pd.read_csv('499.csv')
#df.head()

In [ ]:
#df.to_csv("499.csv", header=['EDAD', 'PESO','TALLA', 'IMC', 'CREATININA', 'CISTATINA', 'HDL', 'LDL', 'TRIGLICERIDOS', 
#'GOT', 'GPT', 'GGT', 'ALBUMINURIA', 'FERRITINA', 'HOMA', 'INSULINA', 'GLUCEMIA', 
#'HB-GLICOSILADA', 'PCR', 'VITAMINA-D', 'TAS', 'TAD', 'FECHA'], index=False)

### CONVERTIR PACIENTES (los 1700 csv) A UNA LISTA Y LA LISTA A CSV

In [7]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
csv_files = glob.glob('*.csv')
csv_files.pop()

new_csv_files =  [item.replace(".csv", "") for item in csv_files]
#new_csv_files.pop()
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(new_csv_files)

['100220', '100257', '100396', '10045', '100505', '100598', '100756', '100889', '101033', '10113', '101310', '101811', '101956', '102007', '102025', '102370', '102634', '102790', '10281', '103000', '103232', '104288', '104376', '104737', '104809', '105227', '105260', '105426', '105617', '10579', '106070', '106107', '106545', '106582', '106810', '106831', '107178', '107918', '107954', '108222', '108390', '108657', '1087', '108789', '109195', '109303', '109322', '109853', '109972', '110337', '110457', '11068', '110788', '11129', '111524', '111529', '111778', '111892', '111907', '112021', '112035', '112225', '112238', '1129', '11300', '11318', '113556', '114131', '114513', '114541', '114611', '114670', '114840', '115087', '115353', '115857', '115994', '116111', '116420', '11667', '116729', '116756', '116788', '116994', '117346', '117851', '11791', '118197', '118206', '118216', '118740', '118769', '119174', '119621', '119972', '120267', '120652', '121079', '121198', '12170', '121800', '121

In [8]:
len(new_csv_files)

1794

In [9]:
df = pd.DataFrame(new_csv_files, columns=["Pacientes"])
df.to_csv('new_csv_files.csv', index=False)

### CREO UNA LISTA A PARTIR DEL CSV PACIENTES PROGRESORES

In [10]:
with open('pacientes_progresores.csv', 'r') as archivo:
    leer = csv.reader(archivo)
    lista = list(leer)
    #print(lista)

In [11]:
lista_HC = []
for i in lista:
    lista_HC.append(i[0])

In [12]:
lista_HC.pop(0)
#print(lista_HC)
print(len(lista_HC))

1647


### COMPARO AMBAS LISTAS Y ELIMINO PACIENTES NO COINCIDENTES

In [13]:
comparacion = []
 
for item in new_csv_files:
    if item in lista_HC:
        comparacion.append(item)
    

In [16]:
print(comparacion)

['100220', '100257', '10045', '100505', '100598', '100756', '100889', '101033', '10113', '101310', '101811', '101956', '102007', '102025', '102370', '102634', '102790', '10281', '103000', '103232', '104288', '104376', '104737', '104809', '105227', '105260', '105426', '105617', '10579', '106070', '106107', '106545', '106810', '106831', '107178', '107918', '107954', '108222', '108390', '108657', '1087', '108789', '109195', '109303', '109322', '109853', '109972', '110337', '110457', '11068', '110788', '11129', '111524', '111529', '111778', '111892', '112021', '112225', '112238', '1129', '11300', '11318', '113556', '114131', '114513', '114541', '114611', '114670', '114840', '115087', '115353', '115857', '115994', '116420', '11667', '116729', '116788', '116994', '117851', '11791', '118197', '118206', '118740', '118769', '119174', '119621', '119972', '120267', '120652', '121079', '121198', '12170', '121800', '121961', '122066', '122138', '122170', '12236', '122688', '122826', '122891', '1229

In [14]:
df = pd.DataFrame(comparacion, columns=["Pacientes"])
df.to_csv('pacientes_procesados.csv', index=False)

### AHORA TENGO UN CSV EN EL CUAL CADA LINEA ES CADA UNO DE LOS CSV INDEPENDIENTES QUE TENIA ANTERIORMENTE PARA CADA SUJETO, ¿COMO PUEDO HACER QUE CADA LINEA VUELVA A SER UN CSV INDEPENDIENTE Y ASI TENER 1647 PACIENTES TRAS HABER ELIMINADO LOS 100 Y PICO QUE YA ERAN DIABÉTICOS?

In [ ]:
from contextlib import ExitStack
import math

# Generamos una función para leer las líneas de 
# un fichero.
def get_lines(fname):
    with open(fname, 'rt') as file:
        lines = 0
        for l in file: lines += 1
    return lines

# Función para dividir el fichero csv en varios
def split_csv(path_csv, splitby):

    with open(path_csv, 'r') as csvfile, ExitStack() as stack:
        # Líneas totales del fichero
        lines = get_lines(path_csv)

        # número de líneas por fichero (excepto el último)
        chunk_num_lines = math.floor(lines/splitby)
        chunk_num_lines = chunk_num_lines if chunk_num_lines > 0 else 1

        # Máximo de ficheros que podemos generar
        # por si ponemos un número "loco"
        max_files = min(lines, splitby)

        # Archivos que vamos a crear
        filenames = ['chunk{}.csv'.format(i) for i in range(0, max_files)]

        # Ficheros (Chunks)
        files = [ stack.enter_context(open(fname, 'wt')) for fname in filenames ]

        # Índice del fihcero actual
        file_index = 0
        # Máximo índice
        max_index = len(filenames) - 1

        # Recorremos el fichero
        for idx, line in enumerate(csvfile):
            # Obtenemos el chunk sobre el que escribir
            # en base al índice actual
            file = files[file_index]
            # Escribimos línea
            file.write(line)

            # Actualizamos el índice cuando hayamos escrito
            # el máximo de líneas para el fichero
            # escepto si file_index == max_index
            if (idx + 1) % chunk_num_lines == 0 and file_index < max_index:
                file_index += 1

In [ ]:
#get_lines('pacientes_procesados.csv')

In [ ]:
#split_csv('pacientes_procesados.csv', 1648)

In [15]:
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes
for filename in new_csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola
#list_data
 
#pd.concat(list_data,axis = 1, sort = False)

FileNotFoundError: [Errno 2] File b'100220' does not exist: b'100220'